This is a quick notebook to just illustrate the usage of LoRA+ using Trainer. Note that it is a very simple finetuning task for the ViT model so there is not much possible performance gain using LoRA+ over LoRA and loraplus_lr_ratio should be set near one. 
- See README for more info about setting loraplus_lr_ratio.
- See the glue folder for examples where LoRA+ leads to real performance gains.

In [1]:
import transformers
import accelerate
import peft

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
model_checkpoint = "google/vit-base-patch16-224-in21k"  # pre-trained model from which to fine-tune

In [4]:
from datasets import load_dataset

dataset = load_dataset("food101", split="train[:5000]")

In [5]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'baklava'

In [6]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [7]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)


def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [8]:
# split up training into training + validation
splits = dataset.train_test_split(test_size=0.1)
train_ds = splits["train"]
val_ds = splits["test"]

In [9]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [11]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
print_trainable_parameters(model)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 85876325 || all params: 85876325 || trainable%: 100.00


In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    use_original_init=False,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 667493 || all params: 86543818 || trainable%: 0.77


In [13]:
from transformers import TrainingArguments, Trainer


model_name = model_checkpoint.split("/")[-1]
batch_size = 128

args = TrainingArguments(
    f"{model_name}-finetuned-lora-food101",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=8,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    seed=0,
    overwrite_output_dir=True,
    push_to_hub=False,
    label_names=["labels"],
)

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [15]:
import torch


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [16]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [17]:
train_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.444604,0.886000
2,2.025100,0.162307,0.960000
3,0.313100,0.152345,0.952000
4,0.203300,0.143446,0.956000
5,0.153700,0.133482,0.964000
6,0.136200,0.132092,0.958000
7,0.118900,0.126595,0.968000
8,0.106400,0.128341,0.962000


Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-lora-food101/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-lora-food101/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-lora-food101/checkpoint-27 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-lora-food101/checkpoint-36 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-lora-food101/checkpoint-45 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [18]:
model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
print_trainable_parameters(model)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 85876325 || all params: 85876325 || trainable%: 100.00


In [19]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    use_original_init=False,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 667493 || all params: 86543818 || trainable%: 0.77


In [20]:
from loraplus import LoraPlusTrainingArguments, LoraPlusTrainer


model_name = model_checkpoint.split("/")[-1]
batch_size = 128

lp_args = LoraPlusTrainingArguments(
    f"{model_name}-finetuned-loraplus-food101",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=8,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    seed=0,
    overwrite_output_dir=True,
    push_to_hub=False,
    label_names=["labels"],
    loraplus_lr_embedding=1e-06,
    loraplus_lr_ratio=1.25,
)

In [21]:
lp_trainer = LoraPlusTrainer(
    model,
    lp_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

In [22]:
lp_train_results = lp_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.727453,0.884000
2,2.364000,0.201306,0.952000
3,0.406400,0.166673,0.948000
4,0.213200,0.139400,0.962000
5,0.165800,0.128099,0.966000
6,0.142700,0.139247,0.968000
7,0.130100,0.129946,0.964000
8,0.112900,0.133401,0.968000


Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-loraplus-food101/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-loraplus-food101/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-loraplus-food101/checkpoint-27 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-loraplus-food101/checkpoint-36 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory vit-base-patch16-224-in21k-finetuned-loraplus-food101/checkpoint-45 already exists and is non-empty.Saving will proceed but saved results may be invalid.
